# 1. Preparación de Entorno

In [1]:
from dotenv import load_dotenv

import os

# Carga las variables de entorno desde el archivo .env
load_dotenv()

bucket = "datasetsaudio"

#Utilitario para manifrompular los servicios de AWS
import boto3

In [2]:
access_key_id = os.getenv('AWS_ACCESS_KEY_ID')
secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')
region = os.getenv('AWS_DEFAULT_REGION')

# 2. Conexión al servicio

In [3]:
#Obtenemos el cliente de servicio
transcribe = boto3.client(
  "transcribe", #Servicio al que nos conectamos
  aws_access_key_id = access_key_id, #Identificador de la clave
  aws_secret_access_key = secret_access_key, #Contraseña de la clave
  region_name = region #Región de la clave
)

In [4]:
#También necesitaremos el cliente del S3
s3 = boto3.client(
  "s3", #Servicio al que nos conectamos
  aws_access_key_id = access_key_id, #Identificador de la clave
  aws_secret_access_key = secret_access_key, #Contraseña de la clave
  region_name = region #Región de la clave
)

# 3. Envío de consulta

In [5]:
#Importamos la librería que manipula la fecha y hora
from datetime import datetime

In [6]:
#Obtenemos la fecha y hora actual
fecha = datetime.today()

In [7]:
# Formateamos la fecha y hora
# Extraemos:
#
# - El año: %Y
# - El mes: %m
# - El día: %d
# - La hora: %H
# - El minuto: %M
# - El segundo: %S
# - El microsegundo: %f
fechaStr = fecha.strftime("%Y%m%d%H%M%S%f")

#Verificamos
fechaStr

'20241021133045607687'

In [8]:
#Colocamos el nombre del JOB de la transcripción
nombreDelJob = f"transcripcion-bda-{fechaStr}"

#Verificamos
print(nombreDelJob)

transcripcion-bda-20241021133045607687


In [9]:
#Enviamos la consulta
#Es una consulta asíncrona
respuesta = transcribe.start_transcription_job(
    TranscriptionJobName = nombreDelJob, #Nombre del JOB
    LanguageCode = "es-ES", #Idioma del audio
    MediaFormat = "wav", #Formato del audio
    Media = {
        "MediaFileUri": f"s3://{bucket}/content/AUDIO_CALL_CENTER.wav", #Ruta del audio
    }
)

# 4. Bucle de espera hasta la finalización del proceso

In [10]:
#Obtenemos el job según su nombre
proceso = transcribe.get_transcription_job(TranscriptionJobName = nombreDelJob)

In [12]:
#Verificamos el estado
proceso["TranscriptionJob"]["TranscriptionJobStatus"]

'IN_PROGRESS'

In [13]:
#Librería para pausar el código
import time 

In [14]:
#Entramos en bucle infinito
#TIEMPO: 1 MINUTO
while True:

  #Obtenemos el job según su nombre
  proceso = transcribe.get_transcription_job(TranscriptionJobName = nombreDelJob)

  #Verificamos el estado
  if proceso["TranscriptionJob"]["TranscriptionJobStatus"] in ["COMPLETED", "FAILED"]:
    #Indicamos la finalización del proceso
    print("Proceso finalizado!")

    #Si finalizó, salimos del bucle infinito
    break
  else:
    #Mostramos el estado del proceso
    print(proceso["TranscriptionJob"]["TranscriptionJobStatus"])

  #Detenemos el código por 5 segundos antes de repetir el bucle
  time.sleep(5)

Proceso finalizado!


# 5. Análisis de la respuesta

In [15]:
#Obtenemos el job según su nombre
proceso = transcribe.get_transcription_job(TranscriptionJobName = nombreDelJob)

#Mostramos el estado del proceso
print(proceso["TranscriptionJob"]["TranscriptionJobStatus"])

COMPLETED


In [16]:
#Vemos el contenido
proceso

{'TranscriptionJob': {'TranscriptionJobName': 'transcripcion-bda-20241021133045607687',
  'TranscriptionJobStatus': 'COMPLETED',
  'LanguageCode': 'es-ES',
  'MediaSampleRateHertz': 48000,
  'MediaFormat': 'wav',
  'Media': {'MediaFileUri': 's3://datasetsaudio/content/AUDIO_CALL_CENTER.wav'},
  'Transcript': {'TranscriptFileUri': 'https://s3.us-east-1.amazonaws.com/aws-transcribe-us-east-1-prod/767828747956/transcripcion-bda-20241021133045607687/ee59fb79-adfb-4dda-8235-38804261a45f/asrOutput.json?X-Amz-Security-Token=IQoJb3JpZ2luX2VjECoaCXVzLWVhc3QtMSJIMEYCIQDlZpUlUsiQC4%2BDMFRTNY72iKXbSoLv2qEdHUy5%2FV8YawIhAPjONylW0HhZxrT8dRCbWWDa4qmqJmxMPBHOsO4lmgIDKroFCJL%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEQBBoMMjc2NjU2NDMzMTUzIgwOBXeJrsqGH4pcuD4qjgX7Pe61aXJ4JjqnxQcrJjhsU7oFhzsbJB1szKcqpzKMb3Eug1RQWRwBgxXskFCY8Ig10H7yxMWcyYx5J4dN01HUU9qB5EvdX%2F3Hqy9hjm4IHrhxDvIxlM%2BCmo5uSbGgQ1JGkoUULieVumWaBohZ%2BBdxo%2FkbGTGajWRGgpvq39HPgpeHYbMQaCMAmX2bvFa759X518T98nP%2BnBrmjEVdBLfMydo70vsDFY%2FxvdYF0iacQy7JAD%2BS4ne

In [17]:
#Definimos la URL en donde se encuentra la transcripción
urlDeTranscripcion = proceso["TranscriptionJob"]["Transcript"]["TranscriptFileUri"]

#Verificamos y entramos a la URL para ver el contenido
print(urlDeTranscripcion)

https://s3.us-east-1.amazonaws.com/aws-transcribe-us-east-1-prod/767828747956/transcripcion-bda-20241021133045607687/ee59fb79-adfb-4dda-8235-38804261a45f/asrOutput.json?X-Amz-Security-Token=IQoJb3JpZ2luX2VjECoaCXVzLWVhc3QtMSJIMEYCIQDlZpUlUsiQC4%2BDMFRTNY72iKXbSoLv2qEdHUy5%2FV8YawIhAPjONylW0HhZxrT8dRCbWWDa4qmqJmxMPBHOsO4lmgIDKroFCJL%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEQBBoMMjc2NjU2NDMzMTUzIgwOBXeJrsqGH4pcuD4qjgX7Pe61aXJ4JjqnxQcrJjhsU7oFhzsbJB1szKcqpzKMb3Eug1RQWRwBgxXskFCY8Ig10H7yxMWcyYx5J4dN01HUU9qB5EvdX%2F3Hqy9hjm4IHrhxDvIxlM%2BCmo5uSbGgQ1JGkoUULieVumWaBohZ%2BBdxo%2FkbGTGajWRGgpvq39HPgpeHYbMQaCMAmX2bvFa759X518T98nP%2BnBrmjEVdBLfMydo70vsDFY%2FxvdYF0iacQy7JAD%2BS4neCQCq%2BxwgMwnThbqQwuV%2FxbdDyPr%2BKJxUrm1vh%2Fjq6TqHEBaqSSWbJGDIUfbrMYLGQecYJ5Zw6X5R%2FXi3%2FqowWO3mX8bSmk0mAZPUz3aJrZ%2BYC%2BkK1RHVVGWOknECgTa%2B3PyWDMEJ8ckR3NkoR%2FPqDms1FwcoYfr5CVZj2Rcu%2FBShbK%2BOvTSIwlv2A7Ob%2BM39VPy%2BNZYCVmuCcgCnvFuNO7KBw0GdVswoYL%2BSvTXRymJSM%2BoWxCf3lcCFJAkM2CznFFL9ejnoSTaJXQAFT6RFr8y%2FKu5TzfdpyWu2NnD3VTO

In [18]:
#Librería para descargar desde enlaces
import requests

In [19]:
    #Librería para manipular JSONs
import json

In [20]:
#Descargamos el archivo
archivo = requests.get(urlDeTranscripcion)

In [21]:
#Convertimos el contenido del archivo en un JSON
archivoJson = archivo.json()

#Verificamos
print(archivoJson)

{'jobName': 'transcripcion-bda-20241021133045607687', 'accountId': '767828747956', 'status': 'COMPLETED', 'results': {'transcripts': [{'transcript': 'Hola, Santiago. Cómo estás? Te habla Andrés, asesor comercial y encargado de tu servicio telefónico. Cómo estás?'}], 'items': [{'id': 0, 'type': 'pronunciation', 'alternatives': [{'confidence': '0.527', 'content': 'Hola'}], 'start_time': '0.009', 'end_time': '0.3'}, {'id': 1, 'type': 'punctuation', 'alternatives': [{'confidence': '0.0', 'content': ','}]}, {'id': 2, 'type': 'pronunciation', 'alternatives': [{'confidence': '0.13', 'content': 'Santiago'}], 'start_time': '0.31', 'end_time': '1.08'}, {'id': 3, 'type': 'punctuation', 'alternatives': [{'confidence': '0.0', 'content': '.'}]}, {'id': 4, 'type': 'pronunciation', 'alternatives': [{'confidence': '0.462', 'content': 'Cómo'}], 'start_time': '1.09', 'end_time': '1.299'}, {'id': 5, 'type': 'pronunciation', 'alternatives': [{'confidence': '0.449', 'content': 'estás'}], 'start_time': '1.30

In [22]:
#Extraemos la transcripción
archivoJson["results"]["transcripts"][0]["transcript"]

'Hola, Santiago. Cómo estás? Te habla Andrés, asesor comercial y encargado de tu servicio telefónico. Cómo estás?'